In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
import pandas as pd
from PIL import Image
import os

In [2]:
# Гиперпараметры
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001
NUM_CLASSES = 50
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Предобработка и загрузка данных
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
train_dir = 'Data/train_butterflies/train_split'
test_dir = 'Data/test_butterflies/valid'

# Тренировочный набор данных
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
# Создание класса Dataset для тестового набора
class TestButterfliesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted([os.path.join(root_dir, fname) for fname in os.listdir(root_dir)])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(image_path)  # Возвращаем имя файла для идентификации

In [12]:
# Загрузка тестового набора данных
test_dataset = TestButterfliesDataset(test_dir, transform=data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
# Загрузка предварительно обученной модели ResNet18 и изменение выходного слоя
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\alexx/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 15.9MB/s]


In [14]:
# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [15]:
# Функция обучения модели
def train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels).item()
            total_predictions += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct_predictions / total_predictions
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

In [16]:
train_model(model, train_loader, criterion, optimizer)

Epoch [1/10], Loss: 1.0439, Accuracy: 0.7221
Epoch [2/10], Loss: 0.5093, Accuracy: 0.8466
Epoch [3/10], Loss: 0.3530, Accuracy: 0.8896
Epoch [4/10], Loss: 0.2787, Accuracy: 0.9169
Epoch [5/10], Loss: 0.2335, Accuracy: 0.9267
Epoch [6/10], Loss: 0.1923, Accuracy: 0.9419
Epoch [7/10], Loss: 0.1540, Accuracy: 0.9524
Epoch [8/10], Loss: 0.1703, Accuracy: 0.9495
Epoch [9/10], Loss: 0.1219, Accuracy: 0.9655
Epoch [10/10], Loss: 0.1164, Accuracy: 0.9651


In [20]:
# Получение предсказаний на тестовом наборе
model.eval()
predictions = []
image_filenames = []

with torch.no_grad():
    for inputs, filenames in test_loader:
        inputs = inputs.to(DEVICE)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        image_filenames.extend(filenames)

# Создание файла для сабмита
submission = pd.DataFrame({'index': range(len(predictions)), 'label': predictions})
submission.to_csv("submission.csv", index=False)